In [ ]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Input, BatchNormalization, Reshape, UpSampling2D, PReLU, ReLU, LeakyReLU, Lambda, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.utils import to_categorical, plot_model
#from tensorflow.keras.applications import ResNet50V2 as pretrained
from tensorflow.keras.applications import Xception as pretrained
#from efficientnet.tfkeras import EfficientNetB0 as pretrained
import tensorflow as tf

import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import cv2
from multiprocessing import Pool
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from timeout_decorator import timeout, TimeoutError

from utils import *

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
noise_length = 128
width = 256
height = 256
epochs = 600
batch_size = 64
n_estimators = 3

In [ ]:
"""
conv_init = RandomNormal(0, 0.02)

def G_model(Height, Width, channel=3):

    inputs = Input((noise_length,))
    x = Dense((Height//16) * (Width//16) * 512)(inputs)
    x = LeakyReLU(0.2)(x)
    x = Reshape(((Height//16) , (Width//16), 512))(x)
    x = Conv2D(1024, 1, strides=1, padding="same",
               kernel_initializer=conv_init)(x)
    x = SubpixelConv2D(x)(x)  # 8x8x256
    x = LeakyReLU(0.2)(x)
    x = Conv2D(512, 1, strides=1, padding="same",
               kernel_initializer=conv_init)(x)
    x = SubpixelConv2D(x)(x)  # 16x16x128
    x = LeakyReLU(0.2)(x)
    x = Conv2D(256, 1, strides=1, padding="same",
               kernel_initializer=conv_init)(x)
    x = SubpixelConv2D(x)(x)  # 32x32x64
    x = LeakyReLU(0.2)(x)
    x = Conv2D(12, 1, strides=1, padding="same",
               kernel_initializer=conv_init)(x)
    x = SubpixelConv2D(x)(x)  # 64x64x3
    outputs = Activation("tanh", name="generator_output")(x)

    model = Model(inputs=inputs, outputs=outputs, name='generator')
    return model
    """


def G_model(Height, Width, channel=3):

    zin = Input((noise_length,))
    z = zin
    for i in range(8):
        z = Dense(512, activation="relu")(z)
    
    c = tf.constant([[1]])
    x = Dense(4*4*512)(c)
    x = Reshape((4, 4, 512))(x)
    
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(256, 3, strides=1, padding="same")(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    
    
    x = UpSampling2D()(x)
    x = Conv2D(128, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(128, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    x = UpSampling2D()(x)
    x = Conv2D(64, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(64, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    x = UpSampling2D()(x)
    x = Conv2D(32, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(32, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    x = UpSampling2D()(x)
    x = Conv2D(16, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(16, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    x = UpSampling2D()(x)
    x = Conv2D(8, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(8, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    
    x = UpSampling2D()(x)
    x = Conv2D(4, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])
    x = Conv2D(3, 3, strides=1, padding="same")(x)
    x = LeakyReLU(0.2)(x)
    x = GaussianNoise(0.1)(x)
    uints, shape = parseshape(x)
    style = Dense(uints)(z)
    style = Reshape(shape)(style)
    x = AdaIN()([x, style])

    outputs = Activation("tanh", name="generator_output")(x)
    model = Model(inputs=zin, outputs=outputs, name="generator")
    return model

In [ ]:
def D_model_core(Height, Width, channel=3):
    inputs = Input((Height, Width, channel))
    x = Conv2D(64, (5, 5), padding="same", strides=(2,2))(inputs)
    x = MaxPooling2D(2)(x)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(128, (5, 5), padding="same", strides=(2,2))(x)
    x = MaxPooling2D(2)(x)
    x = LeakyReLU(0.2)(x)
    x = Conv2D(256, (5, 5), padding="same", strides=(2,2))(x)
    x = MaxPooling2D(2)(x)
    x = LeakyReLU(0.2)(x)
    x = Flatten()(x)
    x = Dropout(0.5) (x)
    x = Dense(1024)(x)
    x = LeakyReLU(0.2)(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inputs, outputs=outputs)
    return model

def D_model(Height, Width, channel=3, n_estimators=3):
    inputs = Input((Height, Width, channel))
    outputs = []
    for i in range(n_estimators):
        outputs.append(D_model_core(Height, Width, channel)(inputs))
    model = Model(inputs=inputs, outputs=outputs, name="discriminator")
    return model

In [ ]:
g = G_model(Height=height, Width=width)
d = D_model(Height=height, Width=width, n_estimators=n_estimators)
#g = load_model("models/generator_256_256_255.h5")
#d = load_model("models/discriminator_256_256_255.h5")

In [ ]:
c = Combined_model(g=g, d=d)

In [ ]:
g_opt = Adam(lr=0.0002, beta_1=0.5)
d_opt = Adam(lr=0.0002, beta_1=0.1)

In [ ]:
g.compile(loss='binary_crossentropy', optimizer='SGD')
d.trainable = False
for layer in d.layers:
    layer.trainable = False
c.compile(loss='binary_crossentropy', optimizer=g_opt)

In [ ]:
d.trainable = True
for layer in d.layers:
    layer.trainable = True
d.compile(loss='binary_crossentropy', optimizer=d_opt)

In [ ]:
g.summary()

In [ ]:
d.summary()

In [ ]:
plot_model(c, show_shapes=True, expand_nested=True)

In [ ]:
def read_file(filepath):
    img = cv2.imread(filepath)
    if img is None:
        print("no such file {0}".format(filepath))
        return
    _width, _height, _ = img.shape
    img = cv2.resize(img, (width,height))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [ ]:
folder="./faces"

X_train = [os.path.join(folder,path) for path in os.listdir(folder)]
with Pool() as p:
    imap = p.imap(read_file, X_train)
    X_train = list(tqdm(imap,total=len(X_train)))

In [ ]:
row = 7
col = 8
plt.figure(figsize=(col*3, row*3))
plt.suptitle("train size: {0}".format(len(X_train)), fontsize=20)
for i in range(row * col):
    plt.subplot(row, col, i+1)
    img = random.choice(X_train)
    plt.imshow(img.astype(np.uint8))
    plt.axis('off')

In [ ]:
g_losses = []
d_losses = []

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    _g_losses = []
    _d_losses = []
    np.random.shuffle(X_train)
    for ite in tqdm(range(1, len(X_train)//batch_size+1)):
        # Discremenator training
        y = X_train[ite * batch_size: (ite+1) * batch_size]
        y += [cv2.flip(img, 1) for img in y]
        y = np.asarray(y)
        y = (y.astype(np.float32)-127.5)/127.5
        input_noise = np.random.uniform(-1, 1, size=(y.shape[0], noise_length))
        g_output = g.predict(input_noise, verbose=0)
        _X_train = np.concatenate((y, g_output))
        _Y_train = [np.array([1] * y.shape[0] + [0] * g_output.shape[0])] * n_estimators
        d_loss = d.train_on_batch(_X_train, _Y_train)
        _d_losses.append(d_loss)
        # Generator training
        #if epoch%3==1:
        input_noise = np.random.uniform(-1, 1, size=(batch_size, noise_length))
        g_loss = c.train_on_batch(input_noise, [[1] * batch_size] * n_estimators)
        _g_losses.append(g_loss)
    
    if epoch%5==1:
        g.save("./models/generator_{0}_{1}_{2}T3.h5".format(width,height,epoch))
        d.save("./models/discriminator_{0}_{1}_{2}T3.h5".format(width,height,epoch))
    g_losses.append(np.mean(_g_losses))
    d_losses.append(np.mean(_d_losses))
    print("g loss = {0}, d loss = {1}".format(g_losses[-1],d_losses[-1]))
    visualize(g,d,epoch,row=1,col=3,save="output_images/epoch{}.png".format(epoch))

In [ ]:
noise = np.random.uniform(-1, 1, size=(1, noise_length))
plt.imshow((g.predict(noise, verbose=0)[0]*127.5+127.5).astype(np.uint8))

In [ ]:
visualize(g,d)

In [ ]:
plt.plot(g_losses)

In [ ]:
plt.plot(d_losses)